# RAGEx for RAG

basierend auf: https://dl.acm.org/doi/pdf/10.1145/3626772.3657660

In [1]:
import sys, json
from pathlib import Path

# Füge das Projektverzeichnis (mit `src/`) dem Python-Pfad hinzu, egal von wo das Notebook gestartet wird.
project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / 'src').exists()), None)
if project_root is None:
    raise RuntimeError("\"src\"-Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.")
root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)


In [ ]:
from src.modules.explainers.rag_ex_explainable import RAGExExplainable
from src.modules.rag.rag_engine import RAGEngine
from src.modules.rag.multihop_rag_engine import MultiHopRAGEngine
from src.modules.llm.llm_client import LLMClient
from src.modules.loader.medmcqa_data_loader import MedMCQADataLoader, format_medmcqa_question
from src.modules.loader.statspearls_data_loader import StatPearlsDataLoader
from src.evaluation.evaluator import Evaluator

import tomllib

### Real data example

In [3]:
config_path = project_root / "config.toml"
config = {}

if config_path.exists():
    with open(config_path, "rb") as f:
        config = tomllib.load(f)

medmcqa_config = config.get("medmcqa") or {}
rag_config = config.get("rag") or {}
llm_config = config.get("llm") or {}

llm_model = llm_config.get("model", "gemma3:4b")
llm_provider = llm_config.get("provider", "ollama")

client = LLMClient(provider=llm_provider, model_name=llm_model)

SPLIT = medmcqa_config.get("split", "val")
PERSIST_DIR = project_root / "data" / "vector_db_statpearls"
NUM_HOPS = rag_config.get('n_hops', 2)

In [4]:
stat_loader = StatPearlsDataLoader(root_dir=str(project_root / "data"))
documents, stats = stat_loader.setup()

rag_engine = RAGEngine(persist_dir=str(PERSIST_DIR))
rag_engine.setup(documents=documents)

multi_hop = MultiHopRAGEngine(rag_engine=rag_engine, llm_client=client, num_hops=NUM_HOPS)

Loading existing vector store from /Users/benediktveith/Documents/Uni/Semester 7/XAI/xai-rag/data/vector_db_statpearls...
RagEngine ready.
Connecting to local Ollama (gemma3:4b)...


In [ ]:
med_loader = MedMCQADataLoader()
questions = med_loader.setup(split=SPLIT, as_documents=False, limit=1)

if not questions:
    raise RuntimeError("No MedMCQA questions loaded.")

results = []
evaluator = Evaluator()
for item in questions:
    question_text = format_medmcqa_question(item)
    if not question_text:
        continue

    trace, _ = multi_hop.run_and_trace(question_text)
    final_answer = (trace.get("final_answer") or "").strip()

    context_blocks = []
    for hop in trace.get("hops", []):
        doc = hop.get("retrieved_document_for_this_hop")
        if doc is None:
            continue
        content = getattr(doc, "page_content", None)
        if content is None:
            content = str(doc)
        context_blocks.append(str(content).strip())

    context = "\n\n".join([c for c in context_blocks if c])
    explainer = RAGExExplainable(llm_client=client)
    explanation = explainer.explain(query=question_text, answer=final_answer, context=context)
    metrics = explainer.metrics()

    perturbed_answers = []
    for result_item in explanation.get("results", []):
        for detail in result_item.get("details", []):
            perturbed_answer = detail.get("perturbed_answer")
            if perturbed_answer:
                perturbed_answers.append(perturbed_answer)

    answer_scores = evaluator.evaluate(perturbed_answers, baseline_answer=final_answer)

    feature_scores = sorted(
        (
            (result_item.get("segment", ""), result_item.get("token", ""), result_item.get("importance", 0.0))
            for result_item in explanation.get("results", [])
        ),
        key=lambda x: x[2],
        reverse=True,
    )

    results.append(
        {
            "question": question_text,
            "final_answer": final_answer,
            "trace": trace,
            "explanation": explanation,
            "metrics": metrics,
            "answer_scores": answer_scores,
            "feature_scores": feature_scores,
        }
    )

print(results[0]["question"])
print(results[0]["final_answer"])
result = results[0]["explanation"]
metrics = results[0]["metrics"]
answer_scores = results[0]["answer_scores"]
feature_scores = results[0]["feature_scores"]


--- Starting Multi-Hop Search for: 'Tensor veli palatini is supplied by:

Options:
A: Facial nerve
B: Trigeminal nerve
C: Glossopharyngeal nerve
D: Pharyngeal plexus' ---

[ Hop 1 ]
Executing search with query: 'Tensor veli palatini is supplied by:

Options:
A: Facial nerve
B: Trigeminal nerve
C: Glossopharyngeal nerve
D: Pharyngeal plexus'
Generating next query...

[ Hop 2 ]
Executing search with query: '“Trigeminal nerve innervation of the tensor veli palatini”'

Generating final answer...

--- Multi-Hop Search Complete. Final Answer: B: Trigeminal nerve ---
--- Multi-Hop Context: 

 ('<doc id="chunk-1-1" from_hop="1" search_query="Tensor veli palatini is supplied by:\n\nOptions:\nA: Facial nerve\nB: Trigeminal nerve\nC: Glossopharyngeal nerve\nD: Pharyngeal plexus">\n[Document(id=\'8c3fde08-f0f6-4e76-b43d-0f2eb9e732c3\', metadata={\'chunk_index\': 14, \'title\': \'Essential organization of the sympathetic nervous system.\', \'source_filename\': \'article-120417.nxml\', \'source\': \

KeyboardInterrupt: 

In [ ]:
print("Explain metrics:", metrics)
print("Answer scores:", answer_scores)
print("Top feature scores:")
for segment, token, score in feature_scores[:10]:
    token_display = str(token).replace("\n", "\\n")
    print(f"{segment}\t{token_display}\t{score:.3f}")

print(RAGExExplainable.prettify(result))


NameError: name 'result' is not defined